In [1]:
import tensorflow.compat.v1 as tf  
tf.disable_v2_behavior()   # tensorflow v2 使用 v1 的语句

import os
import pickle
import numpy as np

"""
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto(allow_soft_placement = True)
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.8)
config.gpu_options.allow_growth = True
"""

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"  #选择哪一块gpu,如果是-1，就是调用cpu
config = tf.ConfigProto() #对session进行参数配置
config.allow_soft_placement=True #如果你指定的设备不存在，允许TF自动分配设备
config.gpu_options.per_process_gpu_memory_fraction=0.7 #分配百分之七十的显存给程序使用，避免内存溢出，可以自己调整
config.gpu_options.allow_growth = True #按需分配显存，这个比较重要

CIFAR_DIR = "./cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

Instructions for updating:
non-resource variables are not supported in the long term
['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
def load_data(filename):
    """read data from data file"""
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']
    
class CifarData:
    def __init__(self, filenames, need_shuffle):
        #训练集需要shuffle，测试集不需要shuffle
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0 #当前到数据集哪个位置
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self, batch_size):
        """return batch_size examples as a batch"""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [3]:
def inception_block(x,
                   output_channel_for_each_path,
                   name):
    """inception block implementation"""
    """
    Args:
    - x:
    - output_channel_for_each_path: eg: [10, 20, 5]
    - name:
    """
    with tf.variable_scope(name):
        conv1_1 = tf.layers.conv2d(x,
                                  output_channel_for_each_path[0],
                                  (1,1),
                                  strides = (1,1),
                                  padding = 'same',
                                  activation = tf.nn.relu,
                                  name = 'conv1_1')
        
        conv3_3 = tf.layers.conv2d(x,
                                  output_channel_for_each_path[1],
                                  (3,3),
                                  strides = (1,1),
                                  padding = 'same',
                                  activation = tf.nn.relu,
                                  name = 'conv3_3')
        
        conv5_5 = tf.layers.conv2d(x,
                                  output_channel_for_each_path[2],
                                  (5,5),
                                  strides = (1,1),
                                  padding = 'same',
                                  activation = tf.nn.relu,
                                  name = 'conv5_5')
        
        max_pooling = tf.layers.max_pooling2d(x,
                                             (2,2), # kernel size
                                             (2,2), # stride
                                             name = 'max_pooling')
        
        max_pooling_shape = max_pooling.get_shape().as_list()[1:]
        input_shape = x.get_shape().as_list()[1:]
        width_padding = (input_shape[0] - max_pooling_shape[0]) // 2
        height_padding = (input_shape[1] - max_pooling_shape[1]) // 2
        padded_pooling = tf.pad(max_pooling,
                               [[0, 0],
                                [width_padding, width_padding],
                                [height_padding, height_padding],
                                [0, 0]]) 
        
        concat_layer = tf.concat([conv1_1, conv3_3, conv5_5, padded_pooling],
                                axis = 3) # 在第4个维度上进行拼接，axis=3
        return concat_layer

x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])

x_image = tf.reshape(x, [-1, 3, 32, 32])
# x_image 32*32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

conv1 = tf.layers.conv2d(x_image,
                        32,
                        (3, 3),
                        padding = 'same',
                        activation = tf.nn.relu,
                        name = 'conv1') # 未定义stride

pooling1 = tf.layers.max_pooling2d(conv1,
                                  (2,2),
                                  (2,2),
                                  name = 'pool1')

inception_2a = inception_block(pooling1,
                              [16, 16, 16],
                              name = 'inception_2a')

inception_2b = inception_block(inception_2a,
                              [16, 16, 16],
                              name = 'inception_2b')

pooling2 = tf.layers.max_pooling2d(inception_2b,
                                  (2,2),
                                  (2,2),
                                  name = 'pool2')

inception_3a = inception_block(pooling2,
                              [16, 16, 16],
                              name = 'inception_3a')

inception_3b = inception_block(inception_3a,
                              [16, 16, 16],
                              name = 'inception_3b')

pooling3 = tf.layers.max_pooling2d(inception_3b,
                                  (2,2),
                                  (2,2),
                                  name = 'pool3')

flatten = tf.layers.flatten(pooling3)
y_ = tf.layers.dense(flatten, 10)

#两类时使用的sigmoid损失函数

"""
p_y_1 = tf.nn.sigmoid(y_)

y_reshape = tf.reshape(y, (-1, 1))
y_reshape_float = tf.cast(y_reshape, tf.float32)
loss = tf.reduce_mean(tf.square(y_reshape_float - p_y_1))

predict = p_y > 0.5
correct_prediction = tf.equal(tf.cast(predict, tf.int64), y_reshape)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))
"""
"""
#多类时使用的softmax损失函数 + mean square loss
p_y = tf.nn.softmax(y_)
# 5 -> [0, 0,0,0,1,0,0,0,0,0]
y_one_hot = tf.one_hot(y, 10, dtype=tf.float32)
y_reshape = tf.reshape(y, (-1, 1))
y_reshape_float = tf.cast(y_reshape, tf.float32)
loss = tf.reduce_mean(tf.square(y_one_hot - p_y))
"""

#交叉熵损失函数
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> softmax
# y -> one_hot
# loss = ylogy_

# indices
predict = tf.argmax(y_, 1)
# 
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('trian_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

E:\Anaconda\envs\tensorflow\lib\site-packages\tensorflow\python\keras\legacy_tf_layers\convolutional.py:414: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  warnings.warn('`tf.layers.conv2d` is deprecated and '
E:\Anaconda\envs\tensorflow\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
E:\Anaconda\envs\tensorflow\lib\site-packages\tensorflow\python\keras\legacy_tf_layers\pooling.py:310: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
  warnings.warn('`tf.layers.max_pooling2d` is deprecated and '
E:\Anaconda\envs\tensorflow\lib\site-packages\tensorflow\python\keras\legacy_tf_layers\core.py:329: UserWarning: `t

In [4]:
init = tf.global_variables_initializer()
batch_size = 10
train_steps = 10000
test_steps = 100

with tf.Session(config=config) as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op], #train_op 加 表示训练
            feed_dict={
                x: batch_data, 
                y: batch_labels})
        if (i+1) % 500 == 0:
            print('[Train] Step : %d, loss: %4.5f, acc: %4.5f'
                 % (i+1, loss_val, acc_val))
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f' % (i+1, test_acc))

[Train] Step : 500, loss: 1.88959, acc: 0.30000
[Train] Step : 1000, loss: 1.46856, acc: 0.60000
[Train] Step : 1500, loss: 0.86938, acc: 0.70000
[Train] Step : 2000, loss: 1.02069, acc: 0.50000
[Train] Step : 2500, loss: 1.25226, acc: 0.60000
[Train] Step : 3000, loss: 1.83909, acc: 0.30000
[Train] Step : 3500, loss: 1.20735, acc: 0.50000
[Train] Step : 4000, loss: 0.85227, acc: 0.80000
[Train] Step : 4500, loss: 0.65162, acc: 0.70000
[Train] Step : 5000, loss: 0.87355, acc: 0.70000
(10000, 3072)
(10000,)
[Test ] Step: 5000, acc: 0.66100
[Train] Step : 5500, loss: 0.94258, acc: 0.70000
[Train] Step : 6000, loss: 0.96073, acc: 0.60000
[Train] Step : 6500, loss: 0.57068, acc: 0.80000
[Train] Step : 7000, loss: 0.49281, acc: 0.90000
[Train] Step : 7500, loss: 1.04917, acc: 0.70000
[Train] Step : 8000, loss: 1.31960, acc: 0.70000
[Train] Step : 8500, loss: 0.62270, acc: 0.80000
[Train] Step : 9000, loss: 0.99358, acc: 0.60000
[Train] Step : 9500, loss: 0.65914, acc: 0.70000
[Train] Step :